In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
# enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-17 02:01:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.50MB/s    in 0.2s    

2020-10-17 02:01:50 (5.50 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
# start a Spark session with an additional option that adds the driver to Spark:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://wine5-bucket.s3.us-east-2.amazonaws.com/Red_Wine_USA.csv"
spark.sparkContext.addFile(url)
Red_Wine_df = spark.read.csv(SparkFiles.get("Red_Wine_USA.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
Red_Wine_df.show()

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120786|Santa Cruz Mountains|  Red|California|    Usa|   2013|      false|95.83|              C+|               8|
|  1|Quintessa, Ruther...| 118360|          Rutherford|  Red|California|    Usa|   2015|      false|95.77|               C|               3|
|  2|Harlan Estate, Na...|  81980|         Napa Valley|  Red|California|    Usa|   2001|      false|98.89|              A+|               6|
|  3|Dalla Valle Viney...|  48290|         Napa Valley|  Red|California|    Usa|   2001|      false|96.49|              B+|               5|
|  4|Philip T

In [6]:
Red_Wine_df.count()

4364

In [5]:
#Drop null values
dropna_df = Red_Wine_df.dropna()
dropna_df.show()

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120786|Santa Cruz Mountains|  Red|California|    Usa|   2013|      false|95.83|              C+|               8|
|  1|Quintessa, Ruther...| 118360|          Rutherford|  Red|California|    Usa|   2015|      false|95.77|               C|               3|
|  2|Harlan Estate, Na...|  81980|         Napa Valley|  Red|California|    Usa|   2001|      false|98.89|              A+|               6|
|  3|Dalla Valle Viney...|  48290|         Napa Valley|  Red|California|    Usa|   2001|      false|96.49|              B+|               5|
|  4|Philip T

In [7]:
dropna_df.count()

4364

In [8]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with journalist_count > 5
cleaned_df = dropna_df.filter(col("journalist_count") > 5)
cleaned_df.show()

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120786|Santa Cruz Mountains|  Red|California|    Usa|   2013|      false|95.83|              C+|               8|
|  2|Harlan Estate, Na...|  81980|         Napa Valley|  Red|California|    Usa|   2001|      false|98.89|              A+|               6|
|  7|Shafer Vineyards,...| 126746| Stags Leap District|  Red|California|    Usa|   2001|      false|93.69|               C|               6|
|  9|Dominus Estate, C...|  67008|         Napa Valley|  Red|California|    Usa|   2001|      false| 93.6|               B|              10|
| 12|Corison,

In [9]:
cleaned_df.count()

446